# CS 447 Homework 3 $-$ Neural Machine Translation
In this homework we are going to perform machine translation using two deep learning approaches: a Recurrent Neural Network (RNN) and Transformer.

Specifically, we are going to train sequence to sequence models for Spanish to English translation. In this assignment you only need to implement the neural network models, we implement all the data loading for you. Please **refer** to the following resources for more details:

1.   https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
2.   https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
3. https://arxiv.org/pdf/1409.0473.pdf

<font color='green'><b>Hint:</b> We suggest that you work on this homework in <b>CPU</b> until you are ready to train. At that point, you should switch your runtime to <b>GPU</b>. You can do this by going to <TT>Runtime > Change Runtime Type</TT> and select "GPU" from the dropdown menu.
* You will find it easier to debug on CPU, and the error messages will be more understandable.
* Google monitors your GPU usage and will occasionally restrict GPU access if you use it too much. In these cases, you can either switch to a different Google account or wait for your access to be restored.</font>

We have imported all the libraries you need to do this homework. <b>You should not import any extra libraries. Furthermore, you should not write any code outside of TODO sections.</b> If you do, the autograder will fail to run your code.

**Reminder: The course policy of this class prohibits the use of AI tools to help with coding or debugging, such as Chat-GPT or Colab's in-built Gemini assistant. Further, you may not look at or copy from code repositories online; and while you may discuss the homework with your classmates, you may *not* share code with each other.** You are of course welcome to look at general Python materials (such as Python or Pytorch tutorials and documentation).


In [ ]:
### DO NOT EDIT ###

import pandas as pd
import unicodedata
import re, math, random, os
from torch.utils.data import Dataset
import torch
rnn_encoder, rnn_encoder, transformer_encoder, transformer_decoder = None, None, None, None
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if __name__=='__main__':
    print('Using device:', DEVICE)

Using device: cuda


# Helper Functions
This cell contains helper functions for the notebook.

In [ ]:
### DO NOT EDIT ###

# Converts the unicode file to ascii
def unicode_to_ascii(s):
    """Normalizes latin chars with accent to their canonical decomposition"""
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    '''
    Preprocess the sentence to add the start, end tokens and make them lower-case
    '''
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r'([?.!,¿])', r' \1 ', w)
    w = re.sub(r'[" "]+', ' ', w)

    w = re.sub(r'[^a-zA-Z?.!,¿]+', ' ', w)

    w = w.rstrip().strip()
    w = '<start> ' + w + ' <end>'
    return w


def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype=np.int64)
    if len(x) > max_len:
        padded[:] = x[:max_len]
    else:
        padded[:len(x)] = x
    return padded


def preprocess_data_to_tensor(dataframe, src_vocab, trg_vocab):
    # Vectorize the input and target languages
    src_tensor = [[src_vocab.word2idx[s if s in src_vocab.vocab else '<unk>'] for s in es.split(' ')] for es in dataframe['es'].values.tolist()]
    trg_tensor = [[trg_vocab.word2idx[s if s in trg_vocab.vocab else '<unk>'] for s in eng.split(' ')] for eng in dataframe['eng'].values.tolist()]

    # Calculate the max_length of input and output tensor for padding
    max_length_src, max_length_trg = max(len(t) for t in src_tensor), max(len(t) for t in trg_tensor)
    print('max_length_src: {}, max_length_trg: {}'.format(max_length_src, max_length_trg))

    # Pad all the sentences in the dataset with the max_length
    src_tensor = [pad_sequences(x, max_length_src) for x in src_tensor]
    trg_tensor = [pad_sequences(x, max_length_trg) for x in trg_tensor]

    return src_tensor, trg_tensor, max_length_src, max_length_trg


def train_test_split(src_tensor, trg_tensor):
    '''
    Create training and test sets.
    '''
    total_num_examples = len(src_tensor) - int(0.2*len(src_tensor))
    src_tensor_train, src_tensor_test = src_tensor[:int(0.75*total_num_examples)], src_tensor[int(0.75*total_num_examples):total_num_examples]
    trg_tensor_train, trg_tensor_test = trg_tensor[:int(0.75*total_num_examples)], trg_tensor[int(0.75*total_num_examples):total_num_examples]

    return src_tensor_train, src_tensor_test, trg_tensor_train, trg_tensor_test

## Sanity Check Function

The code below will be used to perform a sanity check on both the RNN and transformer later in the homework

In [ ]:
### DO NOT EDIT ###

count_parameters = lambda model: sum(p.numel() for p in model.parameters() if p.requires_grad)

def sanityCheckModel(all_test_params, NN, expected_outputs, init_or_forward):
    print('--- TEST: ' + ('Number of Model Parameters (tests __init__(...))' if init_or_forward=='init' else 'Output shape of forward(...)') + ' ---')
    if init_or_forward == "forward":
        # Creating random texts and lables batches
        texts_batch = torch.randint(low=0, high=len(all_test_params[0]['src_vocab']), size=(10,16))
        labels_batch = torch.randint(low=0, high=len(all_test_params[0]['src_vocab']), size=(10,12))

    for tp_idx, (test_params, expected_output) in enumerate(zip(all_test_params, expected_outputs)):
        if init_or_forward == "forward":
            batch_size = test_params['batch_size']
            texts = texts_batch[:batch_size]
            # if NN.__name__ == "RnnEncoder":
            texts = texts.transpose(0,1)

        # Construct the student model
        tps = {k:v for k, v in test_params.items() if k != 'batch_size'}
        stu_nn = NN(**tps)

        input_rep = str({k:v for k,v in tps.items()})

        if init_or_forward == "forward":
            with torch.no_grad():
                if NN.__name__ == "TransformerEncoder":
                    stu_out = stu_nn(texts)
                else:
                    stu_out, _ = stu_nn(texts)
                    expected_output = torch.rand(expected_output).size()
            ref_out_shape = expected_output

            has_passed = torch.is_tensor(stu_out)
            if not has_passed: msg = 'Output must be a torch.Tensor; received ' + str(type(stu_out))
            else:
                has_passed = stu_out.shape == ref_out_shape
                msg = 'Your Output Shape: ' + str(stu_out.shape)


            status = 'PASSED' if has_passed else 'FAILED'
            message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape: ' + str(texts.shape) + '\tExpected Output Shape: ' + str(ref_out_shape) + '\t' + msg
            print(message)
        else:
            stu_num_params = count_parameters(stu_nn)
            ref_num_params = expected_output
            comparison_result = (stu_num_params == ref_num_params)

            status = 'PASSED' if comparison_result else 'FAILED'
            message = '\t' + status + "\tInput: " + input_rep + ('\tExpected Num. Params: ' + str(ref_num_params) + '\tYour Num. Params: '+ str(stu_num_params))
            print(message)

        del stu_nn

## Evaluation Functions

These functions will be used to evaluate both the RNN and Transformer Models.

In [ ]:
### DO NOT EDIT ###

def get_reference_candidate(target, pred, trg_vocab):
    def _to_token(sentence):
        lis = []
        for s in sentence[1:]:
            x = trg_vocab.idx2word[s]
            if x == "<end>": break
            lis.append(x)
        return lis
    reference = _to_token(target)
    candidate = _to_token(pred)
    return reference, candidate

def compute_bleu_scores(target_output, final_output, trg_vocab):
    assert len(target_output) == len(final_output)
    bleu_1 = 0.0
    bleu_2 = 0.0
    bleu_3 = 0.0
    bleu_4 = 0.0

    smoother = SmoothingFunction()
    save_reference = []
    save_candidate = []
    for i in range(len(target_output)):
        reference, candidate = get_reference_candidate(target_output[i], final_output[i], trg_vocab)

        bleu_1 += sentence_bleu(reference, candidate, weights=(1,), smoothing_function=smoother.method1)
        bleu_2 += sentence_bleu(reference, candidate, weights=(1/2, 1/2), smoothing_function=smoother.method1)
        bleu_3 += sentence_bleu(reference, candidate, weights=(1/3, 1/3, 1/3), smoothing_function=smoother.method1)
        bleu_4 += sentence_bleu(reference, candidate, weights=(1/4, 1/4, 1/4, 1/4), smoothing_function=smoother.method1)

        save_reference.append(reference)
        save_candidate.append(candidate)

    bleu_1 = bleu_1/len(target_output)
    bleu_2 = bleu_2/len(target_output)
    bleu_3 = bleu_3/len(target_output)
    bleu_4 = bleu_4/len(target_output)

    scores = {"bleu_1": bleu_1, "bleu_2": bleu_2, "bleu_3": bleu_3, "bleu_4": bleu_4}
    print('BLEU 1-gram: %f' % (bleu_1))
    print('BLEU 2-gram: %f' % (bleu_2))
    print('BLEU 3-gram: %f' % (bleu_3))
    print('BLEU 4-gram: %f' % (bleu_4))

    return save_candidate, scores

# Step 1: Download & Prepare the Data

## Download and Visualize the Data

Here we will download the translation data. We will learn a model to translate Spanish to English.

In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    os.system("wget http://www.manythings.org/anki/spa-eng.zip")
    os.system("unzip -o spa-eng.zip")

Now we view the data.

In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':

    total_num_examples = 50000
    dat = pd.read_csv("spa.txt",
                    sep="\t",
                    header=None,
                    usecols=[0,1],
                    names=['eng', 'es'],
                    nrows=total_num_examples,
                    encoding="UTF-8"
    ).sample(frac=1).reset_index().drop(['index'], axis=1)

    print(dat) # Visualize the data

                            eng                                es
0              I lost the game.                 Perdí la partida.
1       Our team did very well.  Nuestro equipo lo hizo muy bien.
2      Why did you even bother?  ¿Por qué te molestaste siquiera?
3             We met last week.   Nos conocimos la semana pasada.
4       I'll go buy some bread.                Iré a comprar pan.
...                         ...                               ...
49995         I have few books.               Tengo pocos libros.
49996        You are two-faced.                        Sos falso.
49997   Tom is young and naive.         Tomás es joven e ingenuo.
49998            What happened?                   ¿Qué ha pasado?
49999          Let's go inside.               Vamos para adentro.

[50000 rows x 2 columns]


Next we preprocess the data.

In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    data = dat.copy()
    data['eng'] = dat.eng.apply(lambda w: preprocess_sentence(w))
    data['es'] = dat.es.apply(lambda w: preprocess_sentence(w))
    print(data) # Visualizing the data

                                           eng  \
0              <start> i lost the game . <end>   
1       <start> our team did very well . <end>   
2      <start> why did you even bother ? <end>   
3             <start> we met last week . <end>   
4       <start> i ll go buy some bread . <end>   
...                                        ...   
49995         <start> i have few books . <end>   
49996        <start> you are two faced . <end>   
49997   <start> tom is young and naive . <end>   
49998            <start> what happened ? <end>   
49999          <start> let s go inside . <end>   

                                                     es  
0                      <start> perdi la partida . <end>  
1       <start> nuestro equipo lo hizo muy bien . <end>  
2      <start> ¿ por que te molestaste siquiera ? <end>  
3        <start> nos conocimos la semana pasada . <end>  
4                     <start> ire a comprar pan . <end>  
...                                                

## Vocabulary & Dataloader

First we create a class for managing our vocabulary as we did in Homework 2. In this homework, we have a separate class for the vocabulary as we need 2 different vocabularies $-$ one for English and one for Spanish.

Then we prepare the dataloader and make sure it returns the source sentence and target sentence.

In [ ]:
### DO NOT EDIT ###

class Vocab_Lang():
    def __init__(self, vocab):
        self.word2idx = {'<pad>': 0, '<unk>': 1}
        self.idx2word = {0: '<pad>', 1: '<unk>'}
        self.vocab = vocab

        for index, word in enumerate(vocab):
            self.word2idx[word] = index + 2 # +2 because of <pad> and <unk> token
            self.idx2word[index + 2] = word

    def __len__(self):
        return len(self.word2idx)

    def __repr__(self):
        if len(self.vocab) <= 5:
            return str(self.vocab)
        else:
            return f'Vocab_Lang object with {len(self.vocab)} words'

class MyData(Dataset):
    def __init__(self, X, y):
        self.length = torch.LongTensor([np.sum(1 - np.equal(x, 0)) for x in X])
        self.data = torch.LongTensor(X)
        self.target = torch.LongTensor(y)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        return x, y

    def __len__(self):
        return len(self.data)

In [ ]:
### DO NOT EDIT ###

import numpy as np
import random
from torch.utils.data import DataLoader

In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    # HYPERPARAMETERS (You may change these if you want, though you shouldn't need to)
    BATCH_SIZE = 64
    EMBEDDING_DIM = 256

In [ ]:
### DO NOT EDIT ###

def build_vocabulary(pd_dataframe):
    sentences = [sen.split() for sen in pd_dataframe]
    vocab = {}
    for sen in sentences:
        for word in sen:
            if word not in vocab:
                vocab[word] = 1
    return list(vocab.keys())

if __name__ == '__main__':
    src_vocab_list = build_vocabulary(data['es'])
    trg_vocab_list = build_vocabulary(data['eng'])

We instantiate our training and validation datasets.

In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    src_vocab = Vocab_Lang(src_vocab_list)
    trg_vocab = Vocab_Lang(trg_vocab_list)

    src_tensor, trg_tensor, max_length_src, max_length_trg = preprocess_data_to_tensor(data, src_vocab, trg_vocab)
    src_tensor_train, src_tensor_val, trg_tensor_train, trg_tensor_val = train_test_split(src_tensor, trg_tensor)

    # Create train and val datasets
    train_dataset = MyData(src_tensor_train, trg_tensor_train)
    train_dataset = DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)

    test_dataset = MyData(src_tensor_val, trg_tensor_val)
    test_dataset = DataLoader(test_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=False)

max_length_src: 16, max_length_trg: 12


<ipython-input-8-fb8a73e5f1e1>:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  self.data = torch.LongTensor(X)


In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    idxes = random.choices(range(len(train_dataset.dataset)), k=5)
    src, trg =  train_dataset.dataset[idxes]
    print('Source:', src)
    print('Source Dimensions: ', src.size())
    print('Target:', trg)
    print('Target Dimensions: ', trg.size())

Source: tensor([[   2,   14,  375,  281, 1635,  455,   98,   20,    7,    0,    0,    0,
            0,    0,    0,    0],
        [   2,   17,  212,   44,  127,    6,    7,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [   2,   78,   42,  311,  673,    6,    7,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [   2,   17, 4491,   42, 1927,    6,    7,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [   2,  140,  482,   37,   35,    6,    7,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])
Source Dimensions:  torch.Size([5, 16])
Target: tensor([[   2,   14,    5,   96, 1193,   18,    8,    0,    0,    0,    0,    0],
        [   2,    3,   60,  917,   15,    7,    8,    0,    0,    0,    0,    0],
        [   2,  201,   77,   32,  317,  279,    7,    8,    0,    0,    0,    0],
        [   2,    3, 1883,   15,   32, 1876,    7,    8,    0,    0,    0,    0],
        [   2,    3,  138,  409,  131,  

# Step 2: Train a Recurrent Neural Network (RNN) [25 points]

Here you will write a recurrent model for machine translation, and then train and evaluate its results.

Here are some links that you may find helpful:
1. Attention paper: https://arxiv.org/pdf/1409.0473.pdf
2. Explanation of LSTM's & GRU's: https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21
3. Attention explanation: https://towardsdatascience.com/attention-in-neural-networks-e66920838742
4. Another attention explanation: https://towardsdatascience.com/attention-and-its-different-forms-7fc3674d14dc


In [ ]:
### DO NOT EDIT ###

import torch.nn as nn
import torch.nn.functional as F
import time
from tqdm.notebook import tqdm
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction, corpus_bleu

## <font color='red'>TODO:</font> Encoder Model [10 points]

First we build a recurrent encoder model, which will be very similar to what you did in Homework 2. However, instead of using a fully connected layer as the output, you should the return a sequence of outputs of your GRU as well as the final hidden state. These will be used in the decoder.

In this cell, you should implement the `__init(...)` and `forward(...)` functions, each of which is <b>5 points</b>.

In [ ]:
class RnnEncoder(nn.Module):
    def __init__(self, src_vocab, embedding_dim, hidden_units):
        super(RnnEncoder, self).__init__()
        """
        Args:
            src_vocab: Vocab_Lang, the source vocabulary
            embedding_dim: the dimension of the embedding
            hidden_units: The number of features in the GRU hidden state
        """
        self.src_vocab = src_vocab # Do not change
        vocab_size = len(src_vocab)

        ### TODO ###
        # Initialize embedding layer
        # (see: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

        # Initialize a single directional GRU with 1 layer and batch_first=False

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(input_size=embedding_dim,
                          hidden_size=hidden_units,
                          num_layers=1,
                          batch_first=False)

    def forward(self, x):
        """
        Args:
            x: source texts, [max_len, batch_size]

        Returns:
            output: [max_len, batch_size, hidden_units]
            hidden_state: [1, batch_size, hidden_units]

        Pseudo-code:
        - Pass x through an embedding layer and pass the results through the recurrent net
        - Return output and hidden states from the recurrent net
        """
        output, hidden_state = None, None

        ### TODO ###

        embedding = self.embedding(x)
        output, hidden_state = self.gru(embedding)

        return output, hidden_state

The code below runs a sanity check for your `RnnEncoder` class. The tests are similar to the hidden ones in Gradescope. However, note that passing the sanity check does <b>not</b> guarantee that you will pass the autograder; it is intended to help you debug.

In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    # Set random seed
    torch.manual_seed(42)
    # Create test inputs
    embedding_dim = [2, 5, 8]
    hidden_units = [50, 100, 200]
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    params = []
    inputs = []
    for ed in embedding_dim:
        for hu in hidden_units:
            inp = {}
            inp['src_vocab'] = sanity_vocab
            inp['embedding_dim'] = ed
            inp['hidden_units'] = hu
            inputs.append(inp)
    # Test init
    expected_outputs = [8110, 31210, 122410, 8575, 32125, 124225, 9040, 33040, 126040]

    sanityCheckModel(inputs, RnnEncoder, expected_outputs, "init")
    print()

    # Test forward
    inputs = []
    batch_sizes = [1, 2]
    for hu in hidden_units:
        for b in batch_sizes:
            inp = {}
            inp['embedding_dim'] = EMBEDDING_DIM
            inp['src_vocab'] = sanity_vocab
            inp["batch_size"] = b
            inp['hidden_units'] = hu
            inputs.append(inp)
    expected_outputs = [torch.Size([16, 1, 50]), torch.Size([16, 2, 50]), torch.Size([16, 1, 100]), torch.Size([16, 2, 100]), torch.Size([16, 1, 200]), torch.Size([16, 2, 200])]

    sanityCheckModel(inputs, RnnEncoder, expected_outputs, "forward")

--- TEST: Number of Model Parameters (tests __init__(...)) ---
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 2, 'hidden_units': 50}	Expected Num. Params: 8110	Your Num. Params: 8110
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 2, 'hidden_units': 100}	Expected Num. Params: 31210	Your Num. Params: 31210
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 2, 'hidden_units': 200}	Expected Num. Params: 122410	Your Num. Params: 122410
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 5, 'hidden_units': 50}	Expected Num. Params: 8575	Your Num. Params: 8575
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 5, 'hidden_units': 100}	Expected Num. Params: 32125	Your Num. Params: 32125
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 5, 'hidden_units': 200}	Expected Num. Params: 124225	Your Num. Params: 124225
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'hidden_units': 50

## <font color='red'>TODO:</font> Decoder Model [15 points]
We will implement a Decoder model that uses an attention mechanism, as provided in https://arxiv.org/pdf/1409.0473.pdf. We have broken this up into three functions that you need to implement: `__init__(self, ...)`, `compute_attention(self, dec_hs, enc_output)`, and `forward(self, x, dec_hs, enc_output)`:

* <b>`__init__(self, ...)`: [5 points]</b> Instantiate the parameters of your model, and store them in `self` variables.

* <b>`compute_attention(self, dec_hs, enc_output)` [5 points]</b>: Compute the <b>context vector</b>, which is a weighted sum of the encoder output states. Suppose the decoder hidden state at time $t$ is $\mathbf{h}_t$, and the encoder hidden state at time $s$ is $\mathbf{\bar h}_s$. The pseudocode is as follows:

  1. <b>Attention scores:</b> Compute real-valued scores for the decoder hidden state $\mathbf{h}_t$ and each encoder hidden state $\mathbf{\bar h}_s$: $$\mathrm{score}(\mathbf{h}_t, \mathbf{\bar h}_s)=
      \mathbf{v}_a^T \tanh(\mathbf{W}_1 \mathbf{h}_t +\mathbf{W}_2 \mathbf{\bar h}_s)
$$
   Here you should implement the scoring function. A higher score indicates a stronger "affinity" between the decoder state and a specific encoder state.
   
   <font color='green'><b>Hint:</b> the matrices $\mathbf{W}_1$, $\mathbf{W}_2$ and the vector $\mathbf{v_a}$ can all be implemented with `nn.Linear(...)` in Pytorch.</font>

   Note that in theory, $\mathbf{v_a}$ could have a different dimension than $\mathbf{h}_t$ and $\mathbf{\bar h}_s$, but you should use the same hidden size for this vector.

 2. <b>Attention weights:</b> Normalize the attention scores to obtain a valid probability distribution: $$\alpha_{ts} = \frac{\exp \big (\mathrm{score}(\mathbf{h}_t, \mathbf{\bar h}_s) \big)}{\sum_{s'=1}^S \exp \big (\mathrm{score}(\mathbf{h}_t, \mathbf{\bar h}_{s'}) \big)}$$ Notice that this is just the softmax function, and can be implemented with `F.softmax(...)` in Pytorch.

 3. <b>Context vector:</b> Compute a context vector $\mathbf{c}_t$ that is a weighted average of the encoder hidden states, where the weights are given by the attention weights you just computed: $$\mathbf{c}_t=\sum_{s=1}^S \alpha_{ts} \mathbf{\bar h}_s$$

 You should return this context vector, along with the attention weights.



* <b>`forward(self, x, dec_hs, enc_output)`: [5 points]</b> Run a <b>single</b> decoding step, resulting in a distribution over the vocabulary for the next token in the sequence. Pseudocode can be found in the docstrings below.

<font color='green'><b>Hint:</b> You should be able to implement all of this <b>without any for loops</b> using the Pytorch library. Also, remember that these operations should operate in parallel for each item in your batch.</font>

In [ ]:
class RnnDecoder(nn.Module):
    def __init__(self, trg_vocab, embedding_dim, hidden_units):
        super(RnnDecoder, self).__init__()
        """
        Args:
            trg_vocab: Vocab_Lang, the target vocabulary
            embedding_dim: The dimension of the embedding
            hidden_units: The number of features in the GRU hidden state
        """
        self.trg_vocab = trg_vocab # Do not change
        vocab_size = len(trg_vocab)

        ### TODO ###

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.W1 = nn.Linear(hidden_units, hidden_units)
        self.W2 = nn.Linear(hidden_units, hidden_units)
        self.v_a = nn.Linear(hidden_units, 1)

        self.gru = nn.GRU(input_size=embedding_dim + hidden_units,
                          hidden_size=hidden_units,
                          num_layers=1,
                          batch_first=True)

        self.fc = nn.Linear(hidden_units, vocab_size)

    def compute_attention(self, dec_hs, enc_output):
        '''
        This function computes the context vector and attention weights.

        Args:
            dec_hs: Decoder hidden state; [1, batch_size, hidden_units]
            enc_output: Encoder outputs; [max_len_src, batch_size, hidden_units]

        Returns:
            context_vector: Context vector, according to formula; [batch_size, hidden_units]
            attention_weights: The attention weights you have calculated; [batch_size, max_len_src, 1]

        Pseudo-code:
            (1) Compute the attention scores for dec_hs & enc_output
                    - Hint: You may need to permute the dimensions of the tensors in order to pass them through linear layers
                    - Output size: [batch_size, max_len_src, 1]
            (2) Compute attention_weights by taking a softmax over your scores to normalize the distribution (Make sure that after softmax the normalized scores add up to 1)
                    - Output size: [batch_size, max_len_src, 1]
            (3) Compute context_vector from attention_weights & enc_output
                    - Hint: You may find it helpful to use torch.sum & element-wise multiplication (* operator)
            (4) Return context_vector & attention_weights
        '''
        context_vector, attention_weights = None, None

        ### TODO ###

        dec_hs = dec_hs.squeeze(0)

        dec_hs_proj = self.W1(dec_hs).unsqueeze(1)
        enc_output_permute = enc_output.permute(1, 0, 2)
        enc_output_proj = self.W2(enc_output_permute)

        score = self.v_a(torch.tanh(dec_hs_proj + enc_output_proj))

        attention_weights = F.softmax(score, dim=1)


        context_vector = torch.sum(attention_weights * enc_output_permute, dim=1)

        return context_vector, attention_weights



    def forward(self, x, dec_hs, enc_output):
        '''
        This function runs the decoder for a **single** time step.

        Args:
            x: Input token; [batch_size, 1]
            dec_hs: Decoder hidden state; [1, batch_size, hidden_units]
            enc_output: Encoder outputs; [max_len_src, batch_size, hidden_units]

        Returns:
            fc_out: (Unnormalized) output distribution [batch_size, vocab_size]
            dec_hs: Decoder hidden state; [1, batch_size, hidden_units]
            attention_weights: The attention weights you have learned; [batch_size, max_len_src, 1]

        Pseudo-code:
            (1) Compute the context vector & attention weights by calling self.compute_attention(...) on the appropriate input
            (2) Obtain embedding vectors for your input x
                    - Output size: [batch_size, 1, embedding_dim]
            (3) Concatenate the context vector & the embedding vectors along the appropriate dimension
            (4) Feed this result through your RNN (along with the current hidden state) to get output and new hidden state
                    - Output sizes: [batch_size, 1, hidden_units] & [1, batch_size, hidden_units]
            (5) Feed the output of your RNN through linear layer to get (unnormalized) output distribution (don't call softmax!)
            (6) Return this output, the new decoder hidden state, & the attention weights
        '''
        fc_out, attention_weights = None, None

        ### TODO ###
        context_vector, attention_weights = self.compute_attention(dec_hs, enc_output)

        x_embedding = self.embedding(x)

        rnn_input = torch.cat((x_embedding, context_vector.unsqueeze(1)), dim=-1)

        rnn_out, dec_hs = self.gru(rnn_input, dec_hs)

        fc_out = self.fc(rnn_out.squeeze(1))

        return fc_out, dec_hs, attention_weights
    def decode_step(self, inputs, enc_output, dec_hs):
        '''
        Call one step of the decoder.

        Args:
            inputs: Input tokens; [batch_size, sequence length]
            enc_output: Encoder outputs; [max_len_src, batch_size, hidden_units]
            dec_hs: Decoder hidden state; [1, batch_size, hidden_units]

        Returns:
            fc_out: (Unnormalized) output distribution [batch_size, vocab_size]
            dec_hs: Decoder hidden state; [1, batch_size, hidden_units]
        '''
        assert inputs.shape[0] == enc_output.shape[1] == dec_hs.shape[1], 'batch_size must be the same across tensors'
        fc_out, dec_hs, attention_weights = self(inputs[:,-1].unsqueeze(1), dec_hs, enc_output)
        return fc_out, dec_hs

The code below runs a sanity check for your `RnnDecoder` class. The tests are similar to the hidden ones in Gradescope. However, note that passing the sanity check does <b>not</b> guarantee that you will pass the autograder; it is intended to help you debug.

In [ ]:
### DO NOT EDIT ###

def sanityCheckDecoderModelForward(inputs, NN, expected_outputs):
    print('--- TEST: Output shape of forward(...) ---\n')
    expected_fc_outs = expected_outputs[0]
    expected_dec_hs = expected_outputs[1]
    expected_attention_weights = expected_outputs[2]
    msg = ''
    for i, inp in enumerate(inputs):
        input_rep = '{'
        for k,v in inp.items():
            if torch.is_tensor(v):
                input_rep += str(k) + ': ' + 'Tensor with shape ' + str(v.size()) + ', '
            else:
                input_rep += str(k) + ': ' + str(v) + ', '
        input_rep += '}'
        dec = RnnDecoder(trg_vocab=inp['trg_vocab'],embedding_dim=inp['embedding_dim'],hidden_units=inp['hidden_units'])
        dec_hs = torch.rand(1, inp["batch_size"], inp['hidden_units'])
        x = torch.randint(low=0,high=len(inp["trg_vocab"]),size=(inp["batch_size"], 1))
        with torch.no_grad():
            dec_out = dec(x=x, dec_hs=dec_hs,enc_output=inp['encoder_outputs'])
            if not isinstance(dec_out, tuple):
                msg = '\tFAILED\tYour RnnDecoder.forward() output must be a tuple; received ' + str(type(dec_out))
                print(msg)
                continue
            elif len(dec_out)!=3:
                msg = '\tFAILED\tYour RnnDecoder.forward() output must be a tuple of size 3; received tuple of size ' + str(len(dec_out))
                print(msg)
                continue
            stu_fc_out, stu_dec_hs, stu_attention_weights = dec_out
        del dec
        has_passed = True
        msg = ""
        if not torch.is_tensor(stu_fc_out):
            has_passed = False
            msg += '\tFAILED\tOutput must be a torch.Tensor; received ' + str(type(stu_fc_out)) + " "
        if not torch.is_tensor(stu_dec_hs):
            has_passed = False
            msg += '\tFAILED\tDecoder Hidden State must be a torch.Tensor; received ' + str(type(stu_dec_hs)) + " "
        if not torch.is_tensor(stu_attention_weights):
            has_passed = False
            msg += '\tFAILED\tAttention Weights must be a torch.Tensor; received ' + str(type(stu_attention_weights)) + " "

        status = 'PASSED' if has_passed else 'FAILED'
        if not has_passed:
            message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape (x): ' + str(os.XATTR_REPLACE.shape) + '\tExpected Output Shape: ' + str(expected_fc_outs[i]) + '\t' + msg
            print(message)
            continue

        has_passed = stu_fc_out.size() == expected_fc_outs[i]
        msg = 'Your Output Shape: ' + str(stu_fc_out.size())
        status = 'PASSED' if has_passed else 'FAILED'
        message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape (x): ' + str(x.shape) + '\tExpected Output Shape: ' + str(expected_fc_outs[i]) + '\t' + msg
        print(message)

        has_passed = stu_dec_hs.size() == expected_dec_hs[i]
        msg = 'Your Hidden State Shape: ' + str(stu_dec_hs.size())
        status = 'PASSED' if has_passed else 'FAILED'
        message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape (x): ' + str(x.shape) + '\tExpected Hidden State Shape: ' + str(expected_dec_hs[i]) + '\t' + msg
        print(message)

        has_passed = stu_attention_weights.size() == expected_attention_weights[i]
        msg = 'Your Attention Weights Shape: ' + str(stu_attention_weights.size())
        status = 'PASSED' if has_passed else 'FAILED'
        message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape (x): ' + str(x.shape) + '\tExpected Attention Weights Shape: ' + str(expected_attention_weights[i]) + '\t' + msg
        print(message)

        stu_sum = stu_attention_weights.sum(dim=1).squeeze()
        if torch.allclose(stu_sum, torch.ones_like(stu_sum), atol=1e-5):
            print('\tPASSED\t The sum of your attention_weights along dim 1 is 1.')
        else:
            print('\tFAILED\t The sum of your attention_weights along dim 1 is not 1.')
        print()

In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    # Set random seed
    torch.manual_seed(42)
    # Create test inputs
    embedding_dim = [2, 5, 8]
    hidden_units = [50, 100, 200]
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    params = []
    inputs = []
    for ed in embedding_dim:
        for hu in hidden_units:
            inp = {}
            inp['trg_vocab'] = sanity_vocab
            inp['embedding_dim'] = ed
            inp['hidden_units'] = hu
            inputs.append(inp)
    # Test init
    expected_outputs = [21016, 82016, 324016, 21481, 82931, 325831, 21946, 83846, 327646]
    sanityCheckModel(inputs, RnnDecoder, expected_outputs, "init")
    print()

    # Test forward
    inputs = []
    hidden_units = [50, 100, 200]
    batch_sizes = [1, 2, 4]
    embedding_dims = iter([50,80,100,120,150,200,300,400,500])
    encoder_outputs = iter([torch.rand([16, 1, 50]), torch.rand([16, 2, 50]), torch.rand([16, 4, 50]), torch.rand([16, 1, 100]), torch.rand([16, 2, 100]), torch.rand([16, 4, 100]), torch.rand([16, 1, 200]), torch.rand([16, 2, 200]),torch.rand([16, 4, 200])])
    expected_fc_outs = [torch.Size([1, 5]),torch.Size([2, 5]),torch.Size([4, 5]),torch.Size([1, 5]),torch.Size([2, 5]),torch.Size([4, 5]),torch.Size([1, 5]),torch.Size([2, 5]),torch.Size([4, 5])]
    expected_dec_hs = [torch.Size([1, 1, 50]), torch.Size([1, 2, 50]), torch.Size([1, 4, 50]), torch.Size([1, 1, 100]), torch.Size([1, 2, 100]), torch.Size([1, 4, 100]), torch.Size([1, 1, 200]), torch.Size([1, 2, 200]), torch.Size([1, 4, 200])]
    expected_attention_weights = [torch.Size([1, 16, 1]), torch.Size([2, 16, 1]), torch.Size([4, 16, 1]), torch.Size([1, 16, 1]), torch.Size([2, 16, 1]), torch.Size([4, 16, 1]), torch.Size([1, 16, 1]), torch.Size([2, 16, 1]), torch.Size([4, 16, 1])]
    expected_outputs = (expected_fc_outs, expected_dec_hs, expected_attention_weights)

    for hu in hidden_units:
        for b in batch_sizes:
            inp = {}
            edim = next(embedding_dims)
            inp['embedding_dim'] = edim
            inp['trg_vocab'] = sanity_vocab
            inp["batch_size"] = b
            inp['hidden_units'] = hu
            inp['encoder_outputs'] = next(encoder_outputs)
            inputs.append(inp)

    sanityCheckDecoderModelForward(inputs, RnnDecoder, expected_outputs)


--- TEST: Number of Model Parameters (tests __init__(...)) ---
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 2, 'hidden_units': 50}	Expected Num. Params: 21016	Your Num. Params: 21016
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 2, 'hidden_units': 100}	Expected Num. Params: 82016	Your Num. Params: 82016
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 2, 'hidden_units': 200}	Expected Num. Params: 324016	Your Num. Params: 324016
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 5, 'hidden_units': 50}	Expected Num. Params: 21481	Your Num. Params: 21481
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 5, 'hidden_units': 100}	Expected Num. Params: 82931	Your Num. Params: 82931
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 5, 'hidden_units': 200}	Expected Num. Params: 325831	Your Num. Params: 325831
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'hidden_units'

## Train RNN Model

We will train the encoder and decoder using cross-entropy loss.

In [ ]:
### DO NOT EDIT ###

def loss_function(real, pred):
    mask = real.ge(1).float() # Only consider non-zero inputs in the loss

    loss_ = F.cross_entropy(pred, real) * mask
    return torch.mean(loss_)

def train_rnn_model(encoder, decoder, dataset, optimizer, trg_vocab, device, n_epochs):
    batch_size = dataset.batch_size
    for epoch in range(n_epochs):
        start = time.time()
        n_batch = 0
        total_loss = 0

        encoder.train()
        decoder.train()

        for src, trg in tqdm(dataset):
            n_batch += 1
            loss = 0

            enc_output, enc_hidden = encoder(src.transpose(0,1).to(device))
            dec_hidden = enc_hidden

            # use teacher forcing - feeding the target as the next input (via dec_input)
            dec_input = torch.tensor([[trg_vocab.word2idx['<start>']]] * batch_size)

            # run code below for every timestep in the ys batch
            for t in range(1, trg.size(1)):
                predictions, dec_hidden, _ = decoder(dec_input.to(device), dec_hidden.to(device), enc_output.to(device))
                assert len(predictions.shape) == 2 and predictions.shape[0] == dec_input.shape[0] and predictions.shape[1] == len(trg_vocab.word2idx), "First output of decoder must have shape [batch_size, vocab_size], you returned shape " + str(predictions.shape)
                loss += loss_function(trg[:, t].to(device), predictions.to(device))
                dec_input = trg[:, t].unsqueeze(1)

            batch_loss = (loss / int(trg.size(1)))
            total_loss += batch_loss.item()

            optimizer.zero_grad()

            batch_loss.backward()

            ### update model parameters
            optimizer.step()

        ### TODO: Save checkpoint for model (optional)
        print('Epoch:{:2d}/{}\t Loss: {:.4f} \t({:.2f}s)'.format(epoch + 1, n_epochs, total_loss / n_batch, time.time() - start))

    print('Model trained!')

In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    # HYPERPARAMETERS - feel free to change
    LEARNING_RATE = 0.001
    HIDDEN_UNITS=256
    #changed from 10 to 5
    N_EPOCHS=5

    rnn_encoder = RnnEncoder(src_vocab, EMBEDDING_DIM, HIDDEN_UNITS).to(DEVICE)
    rnn_decoder = RnnDecoder(trg_vocab, EMBEDDING_DIM, HIDDEN_UNITS).to(DEVICE)

    rnn_model_params = list(rnn_encoder.parameters()) + list(rnn_decoder.parameters())
    optimizer = torch.optim.Adam(rnn_model_params, lr=LEARNING_RATE)

    print('Encoder and Decoder models initialized!')

Encoder and Decoder models initialized!


In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    train_rnn_model(rnn_encoder, rnn_decoder, train_dataset, optimizer, trg_vocab, DEVICE, N_EPOCHS)

  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 1/5	 Loss: 1.7278 	(18.31s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 2/5	 Loss: 1.0574 	(17.98s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 3/5	 Loss: 0.7426 	(16.82s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 4/5	 Loss: 0.5229 	(15.62s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 5/5	 Loss: 0.3678 	(16.69s)
Model trained!


# Step 3: Inference Functions [40 points]

Now that we have trained the model, we can use it on test data. But first, you must *decode* from the model: given an input (Spanish sentence), what is the most likely output (English sentence)?

Recall that sequence-to-sequence models like RNN's factorize the probability distribution of output sequence $\mathbf{y}$ given input sequence $\mathbf{x}$ as $$P(\mathbf{y} \mid \mathbf{x})=\prod _{t=1}^{|\mathbf{y}|}P(y_t\mid y_0 \cdots y_{t-1}, \mathbf{x})$$

where $y_0$ is the start-of-sentence token.

First, you will explore how to *sample* from this distribution. Then, you will implement a decoding method, which aims to find $\mathrm{argmax}_\mathbf{y} P(\mathbf{y} \mid \mathbf{x})$. Because there are infinitely many sequences $\mathbf{y}$, it is not possible to directly optimize this quantity; thus we resort to *beam search decoding*, a heuristic method.

## <font color='red'>TODO:</font> Temperature Sampling [10 points]

Temperature sampling is a method to sample from $P(\mathbf{y}\mid \mathbf{x})$. Recall that at each timestep, the softmax function $\sigma(\mathbf{z})$ transforms the output logits $\mathbf{z}$ of the final layer of the transformer to a probability distribution. To apply temperature $T$, we use a modified softmax function to produce the distribution: $$P(z_i)=\frac{\exp (z_i\,/\,T)}{\sum _j \exp (z_j\,/\,T)}$$ What is the effect of applying temperature?
* If $T=1$, then this is equivalent to the softmax function $-$ hence, the unmodified probability distributon learned by the transformer.
* As $T \rightarrow \infty$, this approaches the uniform distribution.
* As $T \rightarrow 0$, the distribution becomes increasingly "peaked": that is, the probability of the most likely token increases to $1$.

Implement temperature sampling in the following cell by following the instructions in the docstrings.

In [ ]:
def sample_model(encoder, decoder, src, max_decode_len, temperature):
    """
    Perform temperature sampling of the target sentence for each source sentence in src based on the trained encoder and decoder.

    Args:
        encoder: Your RnnEncoder object
        decoder: Your RnnDecoder object
        src: [max_src_length, batch_size] the source sentences you wish to translate
        max_decode_len: The maximum desired length (int) of your target translated sentences
        temperature: The temperature in temperature sampling

    Returns:
        sentences: A list of length batch_size, where sentences[i] is a list containing vocab indexes of the sampled
            trg sentence for the ith item in the src batch.

    Pseudo-code:
    - Obtain encoder output and hidden state by encoding src sentences
    - For 1 ≤ t < max_decode_len:
        - Obtain dec_input as the best words so far for previous time steps (you can get this from curr_output)
        - Obtain your prediction logits and hidden state by feeding dec_input, enc_output, and dec_hidden to decoder
        - Sample from the distribution over tokens (consider using torch.distributions.Categorical) with temperature
        - Save result in curr_output at index t
    - Truncate each sentence in the batch at EOS before returning them as a list of lists of vocab indexes

    Hints:
    - You should interact with the decoder using its decode_step function.
    """

    # Initialize variables
    trg_vocab = decoder.trg_vocab
    batch_size = src.size(1)
    # curr_output = torch.zeros((batch_size, max_decode_len))
    curr_output = torch.zeros((batch_size, max_decode_len), device=src.device)
    eos_idx = trg_vocab.word2idx['<end>']
    device = src.device
    is_rnn = 'Rnn' in encoder.__class__.__name__
    sentences = []

    # We start the decoding with the start token for each example
    dec_input = torch.tensor([[trg_vocab.word2idx['<start>']]] * batch_size, device=device)
    curr_output[:, 0] = dec_input.squeeze(1)

    # Obtain encoder outputs
    if is_rnn: enc_output, dec_hidden = encoder(src)
    else: enc_output, dec_hidden = encoder(src), None
    enc_output, dec_hidden = enc_output.to(device), dec_hidden.to(device) if dec_hidden is not None else None

    ### TODO ###
    #   At each time step, sample the next token and save it in curr_output

    ### TODO ###
    #   For each item in the batch, truncate the sentence to EOS; if EOS was not predicted, choose sentence up to max_decode_len

    for t in range(1, max_decode_len):
        dec_input = curr_output[:, :t].long()
        dec_output, dec_hidden = decoder.decode_step(dec_input, enc_output, dec_hidden)

        logits = dec_output / temperature
        probs = F.softmax(logits, dim=-1)

        dist = torch.distributions.Categorical(probs)
        token = dist.sample()

        curr_output[:, t] = token

        if torch.all(token == eos_idx):
            break

    for i in range(batch_size):
        sentence = curr_output[i].tolist()
        if eos_idx in sentence:
            sentence = sentence[:sentence.index(eos_idx) + 1]
        sentences.append([int(token) for token in sentence])


    return sentences

You can run the cell below to qualitatively compare some of the sentences your model generates with the some of the correct translations.

In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    rnn_encoder.eval()
    rnn_decoder.eval()
    idxes = random.choices(range(len(test_dataset.dataset)), k=5)
    src, trg =  train_dataset.dataset[idxes]
    results = {}
    for temp in [0.01, 1.0, 2.0]:
        results[temp] = sample_model(rnn_encoder, rnn_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), temp)
    for i in range(len(src)):
        print("Source sentence:\t\t", ' '.join([x for x in [src_vocab.idx2word[j.item()] for j in src[i]] if x != '<pad>']))
        print("Target sentence:\t\t", ' '.join([x for x in [trg_vocab.idx2word[j.item()] for j in trg[i]] if x != '<pad>']))
        for temp in results:
            print("Sampled sentence (T="+str(temp)+"):\t", ' '.join([x for x in [trg_vocab.idx2word[j] for j in results[temp][i]] if x != '<pad>']))
        print("----------------")

Source sentence:		 <start> recogere a tom . <end>
Target sentence:		 <start> i ll pick tom up . <end>
Sampled sentence (T=0.01):	 <start> i ll give tom . <end>
Sampled sentence (T=1.0):	 <start> i didn t tom in my forest . <end>
Sampled sentence (T=2.0):	 <start> any helped to tom . name around . <end>
----------------
Source sentence:		 <start> estan rotos . <end>
Target sentence:		 <start> they re broken . <end>
Sampled sentence (T=0.01):	 <start> they re running . <end>
Sampled sentence (T=1.0):	 <start> they are boring . <end>
Sampled sentence (T=2.0):	 <start> they re flexible tickling right . <end>
----------------
Source sentence:		 <start> ¿ es ella japonesa ? <end>
Target sentence:		 <start> is she japanese ? <end>
Sampled sentence (T=0.01):	 <start> is she japanese ? <end>
Sampled sentence (T=1.0):	 <start> is she japanese ? <end>
Sampled sentence (T=2.0):	 <start> is being gray hatch ? <end>
----------------
Source sentence:		 <start> este esta limpio . <end>
Target sentence

## <font color='red'>TODO:</font> Beam Search Decoding [24 points]

Here, instead of sampling sequences from $P(\mathbf{y}\mid \mathbf{x})$, you will find the maximum *a-posteriori* estimate $\mathbf{\hat y} = \mathrm{argmax}_{\mathbf{y}} P(\mathbf{y}\mid \mathbf{x})$. Because it is not possible to search over all sequences $\mathbf{y}$, we use a heuristic method called beam search decoding.

The idea is as follows: at each timestep, you keep track of the $K$ best hypotheses (in terms of probability) seen up to that time that have not terminated; all terminated hypotheses (those that have predicted the end-of-sentence token) that are encountered should be saved along with their log probability. Once you have proceeded for the maximum number of timesteps, the final hypothesis is the one among the terminated hypotheses that has highest probability.

However, to avoid bias toward shorter hypotheses, we apply a length penalty (described on page 12 of [this paper](https://arxiv.org/pdf/1609.08144)) when selecting the final hypothesis. Specifically, out of the hypotheses that have terminated, we select the one that maximizes the score $$\frac{\log P(\mathbf{y}\mid \mathbf{x})}{lp}$$ where the length penalty $lp$ is defined as $$lp = \Big (\frac{5+t}{5+1}\Big )^\alpha $$ The higher the hyperparameter $\alpha$ is, the more short hypotheses are penalized.

See this [blog post](https://cjlovering.github.io/posts/beam-search/index.html) for a useful description of the algorithm along with visualizations; we encourage you to understand this before attempting to implement it.

You will first implement three helper functions, each of which is worth **5 points**.

In [ ]:
def sort_predictions(predictions):
    '''
    Sort prediction probabilities in descending order, and keep track of which hypothesis and vocab index produced each.

    Args:
        predictions: A tensor of size [beam_size, trg_vocab_size] containing log probabilities.

    Returns:
        probs: A tensor of size beam_size*trg_vocab_size containing all log probabilities sorted in descending order. Each row correponds to a separate
           hypothesis.
        idxes: A tensor of size beam_size*trg_vocab_size, where idxes[i] indicates which index in the vocabulary probs[i] corresponds
           to. Thus, 0 ≤ idxes[i] < trg_vocab_size.
        hypothesis_ids: A tensor of size beam_size*trg_vocab_size where hypothesis_ids[i] indicates which hypothesis probs[i]
           corresonds to. Thus, 0 ≤ hypothesis_ids[i] < beam_size.
    '''
    assert len(predictions.shape) == 2, 'predictions should be a 2d tensor'
    beam_size, trg_vocab_size = predictions.shape[0], predictions.shape[1]
    probs = torch.zeros(beam_size * trg_vocab_size, device=predictions.device)
    idxes = torch.zeros(beam_size * trg_vocab_size, device=predictions.device)
    hypothesis_ids = torch.zeros(beam_size * trg_vocab_size, device=predictions.device)

    ### TODO ###

    predictions_1D = predictions.view(-1)
    probs, sorted_indexes = torch.sort(predictions_1D, descending=True)
    hypothesis_ids = sorted_indexes // trg_vocab_size
    idxes = sorted_indexes % trg_vocab_size


    return probs, idxes, hypothesis_ids

The cell below will provide a sanity check for this function.

In [ ]:
### DO NOT EDIT ###

def sanityCheckSortPredictions():
    predictions_list = [torch.tensor([[-9.3672, -10.1368],[-5.2810, -7.4104],[-6.2810, -8.4154]]),
                        torch.tensor([[-5.2810, -7.4104],[-9.3672, -10.1368],[-6.2810, -8.4154]]),
                        torch.tensor([[-6.2810, -8.4154],[-5.2810, -7.4104],[-9.3672, -10.1368]]),
                        torch.tensor([[-10.4256, -4.9818],[-5.0244, -8.7471],[-7.2406, -6.3092]]),
                        torch.tensor([[-7.5414, -7.9009],[-8.2827, -8.5210],[-9.2406, -9.3092]])]
    expected_outputs = [(torch.tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368]),
                         torch.tensor([0, 0, 1, 1, 0, 1]), torch.tensor([1, 2, 1, 2, 0, 0])),
                        (torch.tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368]),
                         torch.tensor([0, 0, 1, 1, 0, 1]), torch.tensor([0, 2, 0, 2, 1, 1])),
                        (torch.tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368]),
                         torch.tensor([0, 0, 1, 1, 0, 1]), torch.tensor([1, 0, 1, 0, 2, 2])),
                        (torch.tensor([ -4.9818,  -5.0244,  -6.3092,  -7.2406,  -8.7471, -10.4256]),
                         torch.tensor([1, 0, 1, 0, 1, 0]), torch.tensor([0, 1, 2, 2, 1, 0])),
                        (torch.tensor([-7.5414, -7.9009, -8.2827, -8.5210, -9.2406, -9.3092]),
                         torch.tensor([0, 1, 0, 1, 0, 1]), torch.tensor([0, 0, 1, 1, 2, 2]))]

    print("--- TEST: sort_predictions() outputs ---")
    print()
    for i in range(len(predictions_list)):
        probs, idxes, hypotheses_ids = sort_predictions(predictions_list[i])
        if torch.equal(probs, expected_outputs[i][0]):
            status = "PASSED"
        else:
            status = "FAILED"
        message = '\t' + status + "\tInput: Tensor of shape " + str(predictions_list[i].shape) + ('\tExpected probs: ' + str(expected_outputs[i][0]) + '\tYour probs: '+ str(probs))
        print(message)

        if torch.equal(idxes, expected_outputs[i][1]):
            status = "PASSED"
        else:
            status = "FAILED"
        message = '\t' + status + "\tInput: Tensor of shape " + str(predictions_list[i].shape) + ('\tExpected idxes: ' + str(expected_outputs[i][1]) + '\tYour idxes: '+ str(idxes))
        print(message)

        if torch.equal(hypotheses_ids, expected_outputs[i][2]):
            status = "PASSED"
        else:
            status = "FAILED"
        message = '\t' + status + "\tInput: Tensor of shape " + str(predictions_list[i].shape) + ('\tExpected hypotheses_ids: ' + str(expected_outputs[i][2]) + '\tYour hypotheses_ids: '+ str(hypotheses_ids))
        print(message)
        print()

if __name__ == '__main__':
    sanityCheckSortPredictions()

--- TEST: sort_predictions() outputs ---

	PASSED	Input: Tensor of shape torch.Size([3, 2])	Expected probs: tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368])	Your probs: tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368])
	PASSED	Input: Tensor of shape torch.Size([3, 2])	Expected idxes: tensor([0, 0, 1, 1, 0, 1])	Your idxes: tensor([0, 0, 1, 1, 0, 1])
	PASSED	Input: Tensor of shape torch.Size([3, 2])	Expected hypotheses_ids: tensor([1, 2, 1, 2, 0, 0])	Your hypotheses_ids: tensor([1, 2, 1, 2, 0, 0])

	PASSED	Input: Tensor of shape torch.Size([3, 2])	Expected probs: tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368])	Your probs: tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368])
	PASSED	Input: Tensor of shape torch.Size([3, 2])	Expected idxes: tensor([0, 0, 1, 1, 0, 1])	Your idxes: tensor([0, 0, 1, 1, 0, 1])
	PASSED	Input: Tensor of shape torch.Size([3, 2])	Expected hypotheses_ids: tensor([0, 2, 0, 2, 1, 1])	Your hypo

In [ ]:
def find_eos_sentences(probs, idxes, hypothesis_ids, curr_output, eos_idx, beam_size, alpha):
    '''
    Find the sentences that have generated EOS, but only if they are within the top beam_size sentences ranked by log probability.
    Return the sentences (specified by a list of indexes from the vocabulary) that have terminated (generated EOS), and their scores.

    Args:
        probs: A tensor of size num_hyps*trg_vocab_size containing log probabilities (where 1 ≤ num_hyps ≤ beam_size). You may
            assume that this is sorted in descending order.
        idxes: A tensor of size num_hyps*trg_vocab_size where idxes[i] indicates which vocab index probs[i] corresponds to
        hypothesis_ids: A tensor of size num_hyps*trg_vocab_size where hypothesis_ids[i] indicates which hypothesis probs[i] corresponds to
        curr_output: A tensor of size [num_hyps, t-1] containing vocab indexes chosen for each hypothesis up to time t-1.
        eos_idx: The end-of-sentence index in the vocabulary.
        beam_size: The beam size.
        alpha: The alpha used in calculating length penalty.
    Returns:
        eos_sents: A list of lists, where eos_sents[i] is a list containing the vocab indices of the ith sentence.
        eos_probs: A list where eos_probs[i] is the score of eos_sents[i]. The score is the log proabability of the sentence, divided by
            the length penalty, which is described above.
    '''

    assert probs.shape == idxes.shape == hypothesis_ids.shape, 'probs, idxes, and hypothesis_ids must all be the same shape'
    assert alpha >= 0 and eos_idx >= 0 and beam_size >= 1

    eos_probs, eos_sents = [], []

    ### TODO ###
    #    1. Find the indexes of idxes that (1) correspond to predicting EOS and (2) are within the top beam_size results ranked by log probability.
    #         Hint: Make a binary mask that is the same size as probs, where an element is True if it meets these conditions.
    #    2. Calculate the length penalty (do not count BOS or EOS in the length!), and divide the log probabilities by it. These are the scores.
    #    3. Take the scores and hypothesis indexes (in hypothesis_ids) corresponding to the indexes you found in step 1.
    #    4. Create the final sentences by concatenating the chosen hypotheses found so far (hint: use curr_output and the hypothesis indexes you just found)
    #         with the EOS token.

    eos_mask = (idxes == eos_idx)

    top_beam_values, top_beam_indices = torch.topk(probs, beam_size)

    top_beam_mask = torch.tensor([False] * probs.shape[0], dtype=torch.bool, device=probs.device)
    top_beam_mask[top_beam_indices] = True
    final_mask = eos_mask & top_beam_mask

    selected_probs = probs[final_mask]
    masked_hypothesis_ids = hypothesis_ids[final_mask]

    if selected_probs.shape[0] == 0:
      return eos_sents, eos_probs

    selected_hypothesis_ids = []
    for hypothesis in masked_hypothesis_ids:
        selected_hypothesis_ids.append(hypothesis.item())

    sentence_lengths = []

    for hypothesis in selected_hypothesis_ids:
        curr_output_len = len(curr_output[hypothesis])
        sentence_lengths.append(max(curr_output_len - 1, 1))  #-1 for NO EOS
    sentence_lengths = torch.tensor(sentence_lengths, dtype=torch.float32, device=probs.device)

    length_penalty = ((5 + sentence_lengths) / (5 + 1)) ** alpha

    eos_scores = selected_probs / length_penalty

    for hypothesis_id, score in zip(selected_hypothesis_ids, eos_scores):
        sentence = curr_output[hypothesis_id].tolist() + [eos_idx]  #Add back EOS
        eos_sents.append(sentence)
        eos_probs.append(score.item())

    return eos_sents, eos_probs


In [ ]:
def find_non_eos_sentences(probs, idxes, hypothesis_ids, curr_output, eos_idx, beam_size, t):
    '''
    Finds the top beam_size continuations based on their cumulative log probability.

    Args:
        probs: A tensor of size beam_size*trg_vocab_size containing log probabilities. You may assume that this is
           sorted in descending order.
        idxes: A tensor of size beam_size*trg_vocab_size where idxes[i] indicates which vocab index probs[i]
           corresponds to
        hypothesis_ids: A tensor of size beam_size*trg_vocab_size where hypothesis_ids[i] indicates which hypothesis
           probs[i] corresponds to
        curr_output: A tensor of size [beam_size, max_decode_len] containing vocab indexes chosen for each hypothesis.
        eos_idx: The end-of-sentence index in the vocabulary.
        beam_size: The beam size.
        t: The timestep currently being processed.
    Returns:
        new_output: A tensor of size [beam_size, max_decode_len], where the first t columns are filled with the
           vocab indexes found so far for each hypothesis chosen.
        new_probs: A tensor of size beam_size where new_probs[i] contains the cumulative log probability of the
           hypothesis new_output[i]
        next_hyps: A tensor of size beam_size where next_hyps[i] contains the hypothesis index from the previous
           timestep that the selected hypothesis new_output[i] continues. For example, if new_output[i] continues
           hypothesis 4 from curr_output (i.e. it continues curr_output[4]), then next_hyps[i] = 4.
    '''

    assert beam_size == curr_output.shape[0], 'curr_output must have beam_size rows'
    assert probs.shape == idxes.shape == hypothesis_ids.shape
    assert eos_idx >= 0 and beam_size >= 1 and t >= 1

    new_output = torch.zeros_like(curr_output, device=curr_output.device)
    new_probs, next_hyps = torch.zeros(beam_size, device=curr_output.device), torch.zeros(beam_size, device=curr_output.device)

    ### TODO ###
    #    1. Find the vocab indexes (hint: in idxes) and probabilities (hint: in probs) of the top beam_size continuations
    #       that are not EOS, based on their log probabilities.
    #    2. Find the hypothesis indexes that each of these continues (hint: in hypothesis_ids).
    #    3. Set the first t-1 columns of new_output to the existing hypotheses that you have chosen to continue (hint: use
    #       the hypothesis indexes you just found along with curr_output)
    #    4. Set the tth column of new_output to the vocab indexes you have selected for this timestep.

    mask = (idxes != eos_idx)
    non_eos_probs = probs[mask]
    non_eos_idxes = idxes[mask]
    non_eos_hypothesis_ids = hypothesis_ids[mask]

    topk_probs, top_beam_indices = torch.topk(non_eos_probs, beam_size)
    topk_vocab_indices = non_eos_idxes[top_beam_indices]
    topk_hypothesis_ids = non_eos_hypothesis_ids[top_beam_indices]

    for i in range(beam_size):
        for j in range(t):
            new_output[i, j] = curr_output[topk_hypothesis_ids[i], j]

    for i in range(beam_size):
        new_output[i, t] = topk_vocab_indices[i]

    new_probs = topk_probs
    next_hyps = topk_hypothesis_ids

    return new_output, new_probs, next_hyps

Now you code the main beam search function, which is worth **11 points**.

In [ ]:
def beam_decode_model(encoder, decoder, src, max_decode_len, beam_size,alpha):
    """
    Perform beam search for the target sentence for the source sentence in src based on the trained encoder and decoder.

    Args:
        encoder: Your RnnEncoder object
        decoder: Your RnnDecoder object
        src: [max_src_length, 1] the source sentence you wish to translate (note: to simplify this function, we do not batch this)
        max_decode_len: The maximum desired length (int) of your target translated sentences
        beam_size: The beam size
        alpha: The alpha in the length penalty formula

    Returns:
        sentence: A list containing vocab indexes of the best target sentence found.

    Pseudo-code:
    - Obtain encoder output and hidden state by encoding src sentences
    - Expand start-of-sentence token by taking the beam_search continuations that have highest probability.
    - For 2 ≤ t < max_decode_len:
        - Expand each of the previous beam_sarch hypotheses, and compute the cumulative log probability of each possible continuation
        - Save the hypotheses that terminate and their log probabilities if they are in the top beam_size hypotheses by log probability
        - Select the beam_size continuations that are not EOS to proceed with based on their log probabilities, and also track
          which hypothesis they continue
        - For the RNN only (not transformer), choose the hidden states correspoonding to the hypotheses that have been selected to continue
    - Among the terminated sentences, select the one with highest score and return it

    Hints:
    - You should interact with the decoder using its decode_step function.
    - This function will be used for the transformer model as well later in the homework. That model does not have a hidden state, so
      dec_hidden is None.
      Thus, when choosing which hidden state vectors to pass between timesteps, you only need to do this when the variable is_rnn is True.
    """

    # Initialize variables
    trg_vocab = decoder.trg_vocab
    eos_idx = trg_vocab.word2idx['<end>']
    batch_size = src.size(1)
    assert batch_size == 1 # For beam search, we will keep it simple and just do one item at a time.
    device = src.device
    is_rnn = 'Rnn' in encoder.__class__.__name__
    sentence = []

    # For beam search, we initialize our candidate hypotheses as start-of-sentence, each having probability 1 (log probability 0)
    curr_output = torch.zeros((beam_size, max_decode_len),device=device) # For beam search, have to track outputs for each beam
    curr_probs = torch.zeros(beam_size,device=device) # Each candidate hypothesis probability
    completed_hypotheses = []
    completed_scores = []

    # We start the decoding with the start token for each example
    dec_input = torch.tensor([[trg_vocab.word2idx['<start>']]] * batch_size, device=device)
    curr_output[:, 0] = dec_input.squeeze(1)

    # Obtain encoder outputs
    if is_rnn: enc_output, dec_hidden = encoder(src)
    else: enc_output, dec_hidden = encoder(src), None
    enc_output, dec_hidden = enc_output.to(device), dec_hidden.to(device) if dec_hidden is not None else None

    ### TODO ###
    #    1. Feed dec_input into the decoder to get the logits and convert them to log probabilities.
    #    2. Sort the probabilities, and select the top beam_size hypotheses and log probabilities (hint: use
    #       find_non_eos_sentences with t=1)
    #    3. Duplicate the encoder output beam_size times in the batch dimension (hint: see torch.repeat). Expected shape:
    #       [max_src_len, beam_size, hidden_units]
    #    4. For the RNN (i.e. if is_rnn), duplicate the hidden state beam_size times in the batch dimension. Expected
    #       shape: [1, beam_size, hidden_units]

    ### TODO ###
    #    For 2 ≤ t < max_decode_len:
    #        1. Obtain dec_input as the words so far for previous time steps (you can get this from curr_output)
    #        2. Obtain your prediction log probabilities and hidden state by calling the decoder and converting to log probabilities
    #        3. Calculate the cumulative log probabilities by adding the new prediction log probabilities to the cumulative
    #           log probabilities from the previous timestep
    #        4. Call sort_predictions
    #        5. Find the EOS hypotheses and scores and save them in completed_hypotheses and completed_scores
    #        6. Find the non-EOS hypotheses and update curr_output and curr_probs.
    #        7. If is_rnn, select the hidden states corresponding to the hypotheses that have been chosen to continue.

    ### TODO ###
    #    Find completed sentence with highest probability; if there are no completed sentences,
    #      return sentence from curr_output that has highest probability


    logits, dec_hidden = decoder.decode_step(dec_input, enc_output, dec_hidden)
    log_probs = F.log_softmax(logits, dim=-1)

    topk_probs, topk_indices = log_probs.topk(beam_size, dim=-1)

    curr_output, curr_probs, next_hyps = find_non_eos_sentences(topk_probs.squeeze(0), topk_indices.squeeze(0).long(),
                                                                torch.arange(beam_size, device=device, dtype=torch.long),
                                                                curr_output, eos_idx, beam_size, t=1)

    # curr_output[:, 1] = topk_indices.squeeze(0).long()
    enc_output = enc_output.repeat(1, beam_size, 1)

    if is_rnn:
        dec_hidden = dec_hidden.expand(-1, beam_size, -1).contiguous()

    for t in range(2, max_decode_len):
        dec_input = curr_output[:, :t].long()
        logits, dec_hidden = decoder.decode_step(dec_input, enc_output, dec_hidden)
        log_probs = F.log_softmax(logits, dim=-1)

        cumulative_log_probs = curr_probs.unsqueeze(1) + log_probs.view(beam_size, -1)

        probs, idxes, hypothesis_ids = sort_predictions(cumulative_log_probs)

        eos_sents, eos_scores = find_eos_sentences(probs, idxes, hypothesis_ids, curr_output[:, :t], eos_idx, beam_size, alpha)
        completed_hypotheses.extend(eos_sents)
        completed_scores.extend(eos_scores)


        if len(completed_hypotheses) >= beam_size:
            break

        curr_output, curr_probs, next_hypothesis = find_non_eos_sentences(probs, idxes, hypothesis_ids, curr_output, eos_idx, beam_size, t)

        if is_rnn:
            dec_hidden = dec_hidden[:, next_hypothesis.long(), :]

    if completed_hypotheses:
        best_idx = completed_scores.index(max(completed_scores))
        sentence = completed_hypotheses[best_idx]
    else:
        best_idx = curr_probs.argmax().item()
        sentence = curr_output[best_idx].tolist()

    filtered_sentence = []
    for token in sentence:
        if token != 0:
            filtered_sentence.append(token)
    sentence = filtered_sentence

    if eos_idx in sentence:
        sentence = sentence[:sentence.index(eos_idx) + 1]

    return sentence

You can run the cell below to qualitatively compare some of the sentences your model generates with the some of the correct translations.

In [ ]:
 ### DO NOT EDIT ###

if __name__ == '__main__':
    rnn_encoder.eval()
    rnn_decoder.eval()
    for h in range(5): # Do whole thing 5 times here, since doing batch size of 1 for beam search
        idxes = random.choices(range(len(test_dataset.dataset)), k=1)
        src, trg =  train_dataset.dataset[idxes]
        beam_size = 5
        alphas = [0.6, 100]
        beam_result = [beam_decode_model(rnn_encoder, rnn_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), beam_size,alpha=alpha) for alpha in alphas]
        greedy_result = beam_decode_model(rnn_encoder, rnn_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), 1,alpha=0)

        print("Source sentence:\t\t\t\t", ' '.join([x for x in [src_vocab.idx2word[j.item()] for j in src[0]] if x != '<pad>']))
        print("Target sentence:\t\t\t\t", ' '.join([x for x in [trg_vocab.idx2word[j.item()] for j in trg[0]] if x != '<pad>']))
        print("Predicted sentence (greedy search):\t\t", ' '.join([x for x in [trg_vocab.idx2word[j] for j in greedy_result] if x != '<pad>']))
        for i in range(len(alphas)):
            print("Predicted sentence (beam search, alpha="+str(alphas[i])+"):\t", ' '.join([x for x in [trg_vocab.idx2word[j] for j in beam_result[i]] if x != '<pad>']))
        print("----------------")



Source sentence:				 <start> es simple . <end>
Target sentence:				 <start> that s simple . <end>
Predicted sentence (greedy search):		 <start> it s simple . <end>
Predicted sentence (beam search, alpha=0.6):	 <start> it s simple . <end>
Predicted sentence (beam search, alpha=100):	 <start> it s simple . <end>
----------------
Source sentence:				 <start> solo cierra los ojos . <end>
Target sentence:				 <start> just close your eyes . <end>
Predicted sentence (greedy search):		 <start> just play your eyes . <end>
Predicted sentence (beam search, alpha=0.6):	 <start> just read your eyes . <end>
Predicted sentence (beam search, alpha=100):	 <start> just shut up on eyes . <end>
----------------
Source sentence:				 <start> ¿ puedo llamarte manana ? <end>
Target sentence:				 <start> may i call you tomorrow ? <end>
Predicted sentence (greedy search):		 <start> can i call you tomorrow ? <end>
Predicted sentence (beam search, alpha=0.6):	 <start> can i call you tomorrow ? <end>
Predicted sen

## Evaluate RNN Model [8 points]

We provide you with a function to run the test set through the model and calculate BLEU scores. We expect your BLEU scores to satisfy the following conditions:  

*   BLEU-1 > 0.290
*   BLEU-2 > 0.081
*   BLEU-3 > 0.059
*   BLEU-4 > 0.056

Read more about Bleu Score at :

1.   https://en.wikipedia.org/wiki/BLEU
2.   https://www.aclweb.org/anthology/P02-1040.pdf

In [ ]:
### DO NOT EDIT ###

def evaluate_model(encoder, decoder, test_dataset, target_tensor_val, device):
    trg_vocab = decoder.trg_vocab
    batch_size = test_dataset.batch_size
    pad_idx = trg_vocab.word2idx['<pad>']

    encoder.eval()
    decoder.eval()

    final_output, target_output = [], []

    with torch.no_grad():
        for batch, (src, trg) in enumerate(test_dataset):
            sentences = sample_model(encoder,
                                     decoder,
                                     src.transpose(0,1).to(DEVICE),
                                     trg.size(1),
                                     temperature=0.0000001) # Low temperature ==> greedy decoding

            final_output += sentences
            trg_sents = [trg[i,:][trg[i,:] != pad_idx].tolist() for i in range(trg.shape[0])]
            target_output += trg_sents

    # Compute BLEU scores
    return compute_bleu_scores(target_output, final_output, trg_vocab)

if __name__ == '__main__':
    rnn_save_candidate, rnn_scores = evaluate_model(rnn_encoder, rnn_decoder, test_dataset, trg_tensor_val, DEVICE)

BLEU 1-gram: 0.294862
BLEU 2-gram: 0.082672
BLEU 3-gram: 0.060284
BLEU 4-gram: 0.057330


# Step 4: Train a Transformer [35 points]

Here you will write a transformer model for machine translation, and then train and evaluate its results. Here are some helpful links:
<ul>
<li> Original transformer paper: https://arxiv.org/pdf/1706.03762.pdf
<li> Helpful tutorial: http://jalammar.github.io/illustrated-transformer/
<li> Another tutorial: http://peterbloem.nl/blog/transformers
</ul>

In [ ]:
### DO NOT EDIT ###

import math

## <font color='red'>TODO:</font> Positional Embeddings [5 points]

Similar to the RNN, we start with the Encoder model. A key component of the encoder is the Positional Embedding. As we know, word embeddings encode words in such a way that words with similar meaning have similar vectors. Because there are no recurrences in a Transformer, we need a way to tell the transformer the relative position of words in a sentence: so will add a positional embedding to the word embeddings. Now, two words with a similar embedding will both be close in meaning and occur near each other in the sentence.

You will create a positional embedding matrix of size $(max\_len, embed\_dim)$ using the following formulae:
<br>
$\begin{align*} pe[pos,2i] &= \sin \Big (\frac{pos}{10000^{2i/embed\_dim}}\Big )\\pe[pos,2i+1] &= \cos \Big (\frac{pos}{10000^{2i/embed\_dim}}\Big ) \end{align*}$

<font color='green'><b>Hint:</b> You should probably take the logarithm of the denominator to avoid raising $10000$ to an exponent and then exponentiate the result before plugging it into the fraction. This will help you avoid numerical (overflow/underflow) issues.

<font color='green'><b>Hint:</b> We encourage you to try to implement this function with no for loops, which is the general practice (as it is faster). However, since we are using relatively small datasets, you are welcome to do this with for loops if you prefer.

In [ ]:
def create_positional_embedding(max_len, embed_dim):
    '''
    Args:
        max_len: The maximum length supported for positional embeddings
        embed_dim: The size of your embeddings
    Returns:
        pe: [max_len, 1, embed_dim] computed as in the formulae above
    '''

    pe = None

    ### TODO ###

    pe = np.zeros((max_len, embed_dim))
    pos = []

    for i in range(max_len):
        pos.append([i])
    pos = np.array(pos)

    for i in range(embed_dim // 2):
        log_denominator = (2 * i / embed_dim) * np.log(10000)
        denominator = np.exp(log_denominator)
        for pos_idx in range(max_len):

            pe[pos_idx, 2 * i] = np.sin(pos[pos_idx].item() / denominator)
            pe[pos_idx, 2 * i + 1] = np.cos(pos[pos_idx].item() / denominator)

    pe_tensor = torch.tensor(pe)
    pe_tensor = pe_tensor.unsqueeze(1)
    pe_tensor = pe_tensor.float()
    pe = pe_tensor

    return pe


## <font color='red'>TODO:</font> Encoder Model [10 points]

Now you will create the Encoder model for the transformer.

In this cell, you should implement the `__init(...)` and `forward(...)` functions, each of which is <b>5 points</b>.

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, src_vocab, embedding_dim, num_heads,
                 num_layers, dim_feedforward, max_len_src, device, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        self.device = device

        """
        Args:
            src_vocab: Vocab_Lang, the source vocabulary
            embedding_dim: the dimension of the embedding (also the number of expected features for the input of the Transformer)
            num_heads: The number of attention heads
            num_layers: the number of Transformer Encoder layers
            dim_feedforward: the dimension of the feedforward network models in the Transformer
            max_len_src: maximum length of the source sentences
            device: the working device (you may need to map your positional embedding to this device)
            dropout: the dropout to be applied. Default=0.1.
        """
        self.src_vocab = src_vocab  # Do not change
        src_vocab_size = len(src_vocab)

        # Create positional embedding matrix
        self.position_embedding = create_positional_embedding(max_len_src, embedding_dim).to(device)
        self.register_buffer('positional_embedding', self.position_embedding)  # This is not learnable

        ### TODO ###

        # Initialize embedding layer

        # Dropout layer

        # Initialize a nn.TransformerEncoder model (you'll need to use embedding_dim,
        #    num_layers, num_heads, & dim_feedforward here)

        self.embedding = nn.Embedding(src_vocab_size, embedding_dim).to(device)
        self.dropout = nn.Dropout(dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )

        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )


    def make_src_mask(self, src):
        """
        Args:
            src: [max_len, batch_size]
        Returns:
            Boolean matrix of size [batch_size, max_len] indicating which indices are padding
        """
        assert len(src.shape) == 2, 'src must have exactly 2 dimensions'
        src_mask = src.transpose(0, 1) == 0  # Padding index (assuming 0 is padding index)
        return src_mask.to(self.device)  # [batch_size, max_src_len]


    def forward(self, x):
        """
        Args:
            x: [max_len, batch_size]
        Returns:
            output: [max_len, batch_size, embed_dim]
        Pseudo-code (note: x refers to the original input to this function throughout the pseudo-code):
        - Pass x through the word embedding
        - Add positional embedding to the word embedding, then apply dropout
        - Call make_src_mask(x) to compute a mask: this tells us which indexes in x
          are padding, which we want to ignore for the self-attention
        - Call the encoder, with src_key_padding_mask = src_mask
        """

        ### TODO ###
        x = x.to(self.device)
        embedded = self.embedding(x)

        seq_len, batch_size = x.shape
        pos_enc = self.positional_embedding[:seq_len, :].to(self.device)
        embedded = embedded + pos_enc
        embedded = self.dropout(embedded)
        src_mask = self.make_src_mask(x).to(self.device)

        self.transformer_encoder.to(self.device)
        output = self.transformer_encoder(embedded, src_key_padding_mask=src_mask)

        return output.to(self.device)


The code below runs a sanity check for your `TransformerEncoder` class. The tests are similar to the hidden ones in Gradescope. However, note that passing the sanity check does <b>not</b> guarantee that you will pass the autograder; it is intended to help you debug.

In [ ]:
### DO NOT EDIT ###

if __name__=="__main__":
    # Set random seed
    torch.manual_seed(42)
    # Create test inputs
    dimf = [50, 100, 150]
    embedding_dim = [4, 8, 12]
    max_len = 16
    num_layers = iter([1,1,1,2,2,2,3,3,3])
    nheads = iter([1, 1, 1, 1, 2, 2, 2, 4, 4])
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    params = []
    inputs = []
    for df in dimf:
        for ed in embedding_dim:
            inp = {}
            inp['src_vocab'] = sanity_vocab
            inp['embedding_dim'] = ed
            inp['num_heads'] = next(nheads)
            inp['dim_feedforward'] = df
            inp['num_layers'] = next(num_layers)
            inp['max_len_src'] = max_len
            inp['device'] = DEVICE
            inputs.append(inp)
    # Test init
    expected_outputs = [570, 1218, 1994, 2020, 4096, 6428, 4370, 8674, 13362]

    sanityCheckModel(inputs, TransformerEncoder, expected_outputs, "init")

--- TEST: Number of Model Parameters (tests __init__(...)) ---
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 4, 'num_heads': 1, 'dim_feedforward': 50, 'num_layers': 1, 'max_len_src': 16, 'device': device(type='cuda')}	Expected Num. Params: 570	Your Num. Params: 570
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'num_heads': 1, 'dim_feedforward': 50, 'num_layers': 1, 'max_len_src': 16, 'device': device(type='cuda')}	Expected Num. Params: 1218	Your Num. Params: 1218
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 12, 'num_heads': 1, 'dim_feedforward': 50, 'num_layers': 1, 'max_len_src': 16, 'device': device(type='cuda')}	Expected Num. Params: 1994	Your Num. Params: 1994
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 4, 'num_heads': 1, 'dim_feedforward': 100, 'num_layers': 2, 'max_len_src': 16, 'device': device(type='cuda')}	Expected Num. Params: 2020	Your Num. Params: 2020
	PASSED	Input: {'src_vocab': ['a', 'a

In [ ]:
### DO NOT EDIT ###

if __name__=="__main__":
    # Set random seed
    torch.manual_seed(42)
    # Test forward
    inputs = []
    embedding_dims = [32,64,128]
    batch_sizes = [1, 2]
    dimf = 100
    nheads = iter([1,1,2,2,4,4])
    num_layers = iter([1,1,2,2,3,3])
    max_len = 16
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    for ed in embedding_dims:
        for b in batch_sizes:
            inp = {}
            inp['src_vocab'] = sanity_vocab
            inp['embedding_dim'] = ed
            inp['num_heads'] = next(nheads)
            inp['dim_feedforward'] = dimf
            inp['num_layers'] = next(num_layers)
            inp['max_len_src'] = max_len
            inp['device'] = DEVICE
            inp["batch_size"] = b
            inputs.append(inp)
    expected_outputs = [torch.Size([16, 1, 32]), torch.Size([16, 2, 32]), torch.Size([16, 1, 64]), torch.Size([16, 2, 64]), torch.Size([16, 1, 128]), torch.Size([16, 2, 128])]

    sanityCheckModel(inputs, TransformerEncoder, expected_outputs, "forward")

--- TEST: Output shape of forward(...) ---
	PASSED	 Init Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 32, 'num_heads': 1, 'dim_feedforward': 100, 'num_layers': 1, 'max_len_src': 16, 'device': device(type='cuda')}	Forward Input Shape: torch.Size([16, 1])	Expected Output Shape: torch.Size([16, 1, 32])	Your Output Shape: torch.Size([16, 1, 32])
	PASSED	 Init Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 32, 'num_heads': 1, 'dim_feedforward': 100, 'num_layers': 1, 'max_len_src': 16, 'device': device(type='cuda')}	Forward Input Shape: torch.Size([16, 2])	Expected Output Shape: torch.Size([16, 2, 32])	Your Output Shape: torch.Size([16, 2, 32])
	PASSED	 Init Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 64, 'num_heads': 2, 'dim_feedforward': 100, 'num_layers': 2, 'max_len_src': 16, 'device': device(type='cuda')}	Forward Input Shape: torch.Size([16, 1])	Expected Output Shape: torch.Size([16, 1, 64])	Your Output Shape: torch.Size([16, 1, 64])
	PASSED	 Init I

## <font color='red'>TODO:</font> Decoder Model [10 points]
Now we implement a Decoder model. Unlike the RNN, you do not need to explicitly compute inter-attention with the encoder; you will use the nn.TransformerDecoder model, which takes care of this for you.

In this cell, you should implement the `__init(...)` and `forward(...)` functions, each of which is <b>5 points</b>.

In [ ]:
class TransformerDecoder(nn.Module):
    def __init__(self, trg_vocab, embedding_dim, num_heads,
                 num_layers, dim_feedforward, max_len_trg, device, dropout=0.1):
        super(TransformerDecoder, self).__init__()
        self.device = device

        """
        Args:
            trg_vocab: Vocab_Lang, the target vocabulary
            embedding_dim: the dimension of the embedding (also the number of expected features for the input of the Transformer)
            num_heads: The number of attention heads
            num_layers: the number of Transformer Decoder layers
            dim_feedforward: the dimension of the feedforward network models in the Transformer
            max_len_trg: maximum length of the target sentences
            device: the working device (you may need to map your positional embedding to this device)
            dropout: the dropout to be applied. Default=0.1.
        """

        self.trg_vocab = trg_vocab
        trg_vocab_size = len(trg_vocab)

        # Create positional embedding matrix
        self.position_embedding = create_positional_embedding(max_len_trg, embedding_dim).to(device)
        self.register_buffer('positional_embedding', self.position_embedding)  # Inform PyTorch that it's not a learnable parameter


        ### TODO ###

        # Initialize embedding layer

        # Dropout layer

        # Initialize a nn.TransformerDecoder model (you'll need to use embedding_dim,
        # num_layers, num_heads, & dim_feedforward here)

        # Final fully connected layer

        self.embedding = nn.Embedding(trg_vocab_size, embedding_dim).to(device)
        self.dropout = nn.Dropout(dropout)
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=embedding_dim,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers).to(device)
        self.fc_out = nn.Linear(embedding_dim, trg_vocab_size).to(device)

    def generate_square_subsequent_mask(self, sz):
        """Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
        """
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).to(self.device)
        return mask

    def forward(self, dec_in, enc_out):
        """
        Args:
            dec_in: [sequence length, batch_size]
            enc_out: [max_len_src, batch_size, embed_dim]
        Returns:
            output: [sequence length, batch_size, trg_vocab_size]
        Pseudo-code:
            - Compute input word and positional embeddings in similar manner to encoder
            - Call generate_square_subsequent_mask() to compute a mask: this time,
              the mask is to prevent the decoder from attending to tokens in the "future".
              In other words, at time step i, the decoder should only attend to tokens
              1 to i-1.
            - Call the decoder, with tgt_mask = trg_mask
            - Run the output through the fully-connected layer and return it
        """
        output = None

        ### TODO ###

        dec_in = dec_in.to(self.device)
        enc_out = enc_out.to(self.device)
        word_embeddings = self.embedding(dec_in)

        seq_len = dec_in.size(0)
        positional_embeddings = self.position_embedding[:seq_len, :]
        x = word_embeddings + positional_embeddings
        x = self.dropout(x)
        tgt_mask = self.generate_square_subsequent_mask(seq_len)
        output = self.transformer_decoder(x, enc_out, tgt_mask=tgt_mask)
        output = self.fc_out(output)

        return output.to(self.device)

    def decode_step(self, inputs, enc_output, dec_hs):
        '''
        Call one step of the decoder.

        Args:
            inputs: Input tokens; [batch_size, sequence length]
            enc_output: Encoder outputs; [max_len_src, batch_size, embed_dim]
            dec_hs: None

        Returns:
            fc_out: (Unnormalized) output distribution [batch_size, vocab_size]
            dec_hs: None
        '''
        assert dec_hs is None, 'For the transformer model, make sure you pass dec_hs = None!'
        preds = self(inputs.transpose(0, 1), enc_output)[-1]
        return preds, None


The code below runs a sanity check for your `TransformerDecoder` class. The tests are similar to the hidden ones in Gradescope. However, note that passing the sanity check does <b>not</b> guarantee that you will pass the autograder; it is intended to help you debug.

In [ ]:
### DO NOT EDIT ###

def sanityCheckTransformerDecoderModelForward(inputs, NN, expected_outputs):
    print('--- TEST: Output shape of forward(...) ---\n')
    msg = ''
    for i, inp in enumerate(inputs):
        input_rep = '{'
        for k,v in inp.items():
            if torch.is_tensor(v):
                input_rep += str(k) + ': ' + 'Tensor with shape ' + str(v.size()) + ', '
            else:
                input_rep += str(k) + ': ' + str(v) + ', '
        input_rep += '}'
        dec = NN(trg_vocab=inp['trg_vocab'],embedding_dim=inp['embedding_dim'],num_heads=inp['num_heads'],num_layers=inp['num_layers'],dim_feedforward=inp['dim_feedforward'],max_len_trg=inp['max_len_trg'],device=inp['device'])
        dec_in = torch.randint(low=0,high=len(inputs[0]['trg_vocab']),size=(inp['max_len_trg'], inp['batch_size']))
        enc_out = torch.rand(inp['max_len_trg'], inp['batch_size'], inp['embedding_dim'])
        inp['encoder_outputs'] = enc_out
        with torch.no_grad():
            stu_out = dec(enc_out=enc_out, dec_in=dec_in)
        del dec
        has_passed = True
        if not torch.is_tensor(stu_out):
            has_passed = False
            msg = 'Output must be a torch.Tensor; received ' + str(type(stu_out))
        status = 'PASSED' if has_passed else 'FAILED'
        if not has_passed:
            message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape (dec_in): ' + str(dec_in.shape) + '\tExpected Output Shape: ' + str(expected_outputs[i]) + '\t' + msg
            print(message)
            continue

        has_passed = stu_out.size() == expected_outputs[i]
        msg = 'Your Output Shape: ' + str(stu_out.size())
        status = 'PASSED' if has_passed else 'FAILED'
        message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape (dec_in): ' + str(dec_in.shape) + '\tExpected Output Shape: ' + str(expected_outputs[i]) + '\t' + msg
        print(message)



In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    # Set random seed
    torch.manual_seed(42)
    # Create test inputs
    hidden_units = [50, 100, 200]
    embedding_dim = [8, 16]
    num_heads = [1, 2]
    dim_feedforward = [50, 100]
    num_layers = [1, 2]
    max_lens = 64
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    params = []
    inputs = []
    for ed in embedding_dim:
        for df in dim_feedforward:
            for nh in num_heads:
                for nl in num_layers:
                    inp = {}
                    inp['trg_vocab'] = sanity_vocab
                    inp['embedding_dim'] = ed
                    inp['num_heads'] = nh
                    inp['num_layers'] = nl
                    inp['dim_feedforward'] = df
                    inp['max_len_trg'] = max_lens
                    inp['device'] = DEVICE
                    inputs.append(inp)
    # Test init
    expected_outputs = [1567, 3049, 1567, 3049, 2417, 4749, 2417, 4749]
    sanityCheckModel(inputs, TransformerDecoder, expected_outputs, "init")
    print()

    # Test forward
    inputs = []
    batch_sizes = [1, 2, 4]
    num_heads = 2
    num_layers = 1
    embedding_dims = iter([100, 100, 200, 200, 200, 400, 400, 800, 800])
    max_lens = iter([16, 16, 16, 32, 32, 32, 64, 64, 128])
    expected_outputs = [torch.Size([16, 1, 5]),torch.Size([16, 2, 5]),torch.Size([16, 4, 5]),torch.Size([32, 1, 5]),torch.Size([32, 2, 5]),torch.Size([32, 4, 5]),torch.Size([64, 1, 5]),torch.Size([64, 2, 5]),torch.Size([128, 4, 5])]

    for hu in hidden_units:
        for b in batch_sizes:
            inp = {}
            edim = next(embedding_dims)
            inp['embedding_dim'] = edim
            inp['trg_vocab'] = sanity_vocab
            inp['num_heads'] = num_heads
            inp['num_layers'] = num_layers
            inp["batch_size"] = b
            inp['dim_feedforward'] = hu
            inp['max_len_trg'] = next(max_lens)
            inp['device'] = DEVICE
            inputs.append(inp)

    sanityCheckTransformerDecoderModelForward(inputs, TransformerDecoder, expected_outputs)


--- TEST: Number of Model Parameters (tests __init__(...)) ---
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'num_heads': 1, 'num_layers': 1, 'dim_feedforward': 50, 'max_len_trg': 64, 'device': device(type='cuda')}	Expected Num. Params: 1567	Your Num. Params: 1567
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'num_heads': 1, 'num_layers': 2, 'dim_feedforward': 50, 'max_len_trg': 64, 'device': device(type='cuda')}	Expected Num. Params: 3049	Your Num. Params: 3049
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'num_heads': 2, 'num_layers': 1, 'dim_feedforward': 50, 'max_len_trg': 64, 'device': device(type='cuda')}	Expected Num. Params: 1567	Your Num. Params: 1567
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'num_heads': 2, 'num_layers': 2, 'dim_feedforward': 50, 'max_len_trg': 64, 'device': device(type='cuda')}	Expected Num. Params: 3049	Your Num. Params: 3049
	PASSED	Input: {'trg_vocab': ['a', 'a

## Train Transformer Model

Like the RNN, we train the encoder and decoder using cross-entropy loss.

In [ ]:
### DO NOT EDIT ###

def train_transformer_model(encoder, decoder, optimizer, device, n_epochs):
    encoder.train()
    decoder.train()
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    for epoch in range(n_epochs):
        start = time.time()
        losses = []

        for src, trg in tqdm(train_dataset):

            src = src.to(device).transpose(0,1) # [max_src_length, batch_size]
            trg = trg.to(device).transpose(0,1) # [max_trg_length, batch_size]

            enc_out = encoder(src)
            output = decoder(trg[:-1, :], enc_out)

            output = output.reshape(-1, output.shape[2])
            trg = trg[1:].reshape(-1)

            optimizer.zero_grad()

            loss = criterion(output, trg)
            losses.append(loss.item())

            loss.backward()

            # Clip to avoid exploding grading issues
            torch.nn.utils.clip_grad_norm_(encoder.parameters(), max_norm=1)
            torch.nn.utils.clip_grad_norm_(decoder.parameters(), max_norm=1)

            optimizer.step()

        mean_loss = sum(losses) / len(losses)
        print('Epoch:{:2d}/{}\t Loss:{:.4f} ({:.2f}s)'.format(epoch + 1, n_epochs, mean_loss, time.time() - start))


In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    # HYPERPARAMETERS - feel free to change
    LEARNING_RATE = 0.001
    DIM_FEEDFORWARD=512
    N_EPOCHS=5 #changed to 5
    N_HEADS=2
    N_LAYERS=2
    DROPOUT=0.1

    transformer_encoder = TransformerEncoder(src_vocab, EMBEDDING_DIM, N_HEADS,
                                 N_LAYERS,DIM_FEEDFORWARD,
                                 max_length_src, DEVICE, DROPOUT).to(DEVICE)
    transformer_decoder = TransformerDecoder(trg_vocab, EMBEDDING_DIM, N_HEADS,
                              N_LAYERS,DIM_FEEDFORWARD,
                              max_length_trg, DEVICE, DROPOUT).to(DEVICE)

    transformer_model_params = list(transformer_encoder.parameters()) + list(transformer_decoder.parameters())
    optimizer = torch.optim.Adam(transformer_model_params, lr=LEARNING_RATE)

    print('Encoder and Decoder models initialized!')

Encoder and Decoder models initialized!


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    train_transformer_model(transformer_encoder, transformer_decoder, optimizer, DEVICE, N_EPOCHS)

  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 1/5	 Loss:2.9458 (10.03s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 2/5	 Loss:1.8676 (9.83s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 3/5	 Loss:1.4129 (9.60s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 4/5	 Loss:1.1353 (9.56s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 5/5	 Loss:0.9523 (9.93s)


## Inference

Now that we have trained the model, we can use it on test data. Since you have already written the sampling and decoding function, we can call them on the transformers.

You can run the cell below to qualitatively compare some of the sentences your model generates with the some of the correct translations.

In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    transformer_encoder.eval()
    transformer_decoder.eval()
    idxes = random.choices(range(len(test_dataset.dataset)), k=5)
    src, trg =  train_dataset.dataset[idxes]
    results = {}
    for temp in [0.01, 1.0, 2.0]:
        results[temp] = sample_model(transformer_encoder, transformer_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), temp)
    for i in range(len(src)):
        print("Source sentence:\t\t", ' '.join([x for x in [src_vocab.idx2word[j.item()] for j in src[i]] if x != '<pad>']))
        print("Target sentence:\t\t", ' '.join([x for x in [trg_vocab.idx2word[j.item()] for j in trg[i]] if x != '<pad>']))
        for temp in results:
            print("Sampled sentence (T="+str(temp)+"):\t", ' '.join([x for x in [trg_vocab.idx2word[j] for j in results[temp][i]] if x != '<pad>']))
        print("----------------")

Source sentence:		 <start> voy a coger la escoba . <end>
Target sentence:		 <start> i ll go get the broom . <end>
Sampled sentence (T=0.01):	 <start> i ll get the broom . <end>
Sampled sentence (T=1.0):	 <start> i m going to size . <end>
Sampled sentence (T=2.0):	 <start> i go to cash admire . <end>
----------------
Source sentence:		 <start> iba a llamar a tom . <end>
Target sentence:		 <start> i was going to call tom . <end>
Sampled sentence (T=0.01):	 <start> i was going to call tom . <end>
Sampled sentence (T=1.0):	 <start> i came for call tom . <end>
Sampled sentence (T=2.0):	 <start> i went lost waiting history tom . <end>
----------------
Source sentence:		 <start> el reconocio su derrota . <end>
Target sentence:		 <start> he admitted his defeat . <end>
Sampled sentence (T=0.01):	 <start> he admitted his defeat . <end>
Sampled sentence (T=1.0):	 <start> he admitted his defeat . <end>
Sampled sentence (T=2.0):	 <start> handed , prices taking . <end>
----------------
Source senten

In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    transformer_encoder.eval()
    transformer_decoder.eval()
    for h in range(5): # Do whole thing 5 times here, since doing batch size of 1 for beam search
        idxes = random.choices(range(len(test_dataset.dataset)), k=1)
        src, trg =  train_dataset.dataset[idxes]
        beam_size = 5
        alphas = [0.6, 100]
        beam_result = [beam_decode_model(transformer_encoder, transformer_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), beam_size,alpha=alpha) for alpha in alphas]
        greedy_result = beam_decode_model(transformer_encoder, transformer_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), 1,alpha=0)

        print("Source sentence:\t\t\t\t", ' '.join([x for x in [src_vocab.idx2word[j.item()] for j in src[0]] if x != '<pad>']))
        print("Target sentence:\t\t\t\t", ' '.join([x for x in [trg_vocab.idx2word[j.item()] for j in trg[0]] if x != '<pad>']))
        print("Predicted sentence (greedy search):\t\t", ' '.join([x for x in [trg_vocab.idx2word[j] for j in greedy_result] if x != '<pad>']))
        for i in range(len(alphas)):
            print("Predicted sentence (beam search, alpha="+str(alphas[i])+"):\t", ' '.join([x for x in [trg_vocab.idx2word[j] for j in beam_result[i]] if x != '<pad>']))
        print("----------------")

Source sentence:				 <start> nunca jugue golf . <end>
Target sentence:				 <start> i ve never played golf . <end>
Predicted sentence (greedy search):		 <start> i never played golf . <end>
Predicted sentence (beam search, alpha=0.6):	 <start> i never played golf . <end>
Predicted sentence (beam search, alpha=100):	 <start> i never played golf . <end>
----------------
Source sentence:				 <start> estoy un poco cansada . <end>
Target sentence:				 <start> i m a bit tired . <end>
Predicted sentence (greedy search):		 <start> i m a little tired . <end>
Predicted sentence (beam search, alpha=0.6):	 <start> i m a little tired . <end>
Predicted sentence (beam search, alpha=100):	 <start> i m a little tired . <end>
----------------
Source sentence:				 <start> huele esto . <end>
Target sentence:				 <start> smell this . <end>
Predicted sentence (greedy search):		 <start> this smells delicious . <end>
Predicted sentence (beam search, alpha=0.6):	 <start> this smells delicious . <end>
Predicted s

## Evaluate Transformer Model [8 points]

Now we can run the test set through the transformer model. We expect your BLEU scores to satisfy the following conditions:

*   BLEU-1 > 0.290
*   BLEU-2 > 0.081
*   BLEU-3 > 0.059
*   BLEU-4 > 0.056


In [ ]:
### DO NOT EDIT ###

if __name__ == '__main__':
    transformer_save_candidate, transformer_scores = evaluate_model(transformer_encoder, transformer_decoder, test_dataset, trg_tensor_val, DEVICE)

BLEU 1-gram: 0.304369
BLEU 2-gram: 0.085991
BLEU 3-gram: 0.063225
BLEU 4-gram: 0.060445


# What to Submit

To submit the assignment, download this notebook as a <TT>.py</TT> file. You can do this by going to <TT>File > Download > Download .py</TT>. Then rename it to `hwk3.py`.

You will also need to save the `rnn_encoder`, `rnn_decoder`, `transformer_encoder` and `transformer_decoder`. You can run the cell below to do this. After you save the files to your Google Drive, you need to manually download the files to your computer, and then submit them to the autograder.

You will submit the following files to the autograder:
1.   `hwk3.py`, the download of this notebook as a `.py` file (**not** a `.ipynb` file)
1.   `rnn_encoder.pt`, the saved version of your `rnn_encoder`
1.   `rnn_decoder.pt`, the saved version of your `rnn_decoder`
1.   `transformer_encoder.pt`, the saved version of your `transformer_encoder`
1.   `transformer_decoder.pt`, the saved version of your `transformer_decoder`

**Reminder: Make sure that you access the Gradescope submission page via the corresponding assignment in Coursera!** Failure to do so may result in the inability to push your grades to Coursera. (The same goes for quizzes!)

In [ ]:
### DO NOT EDIT ###

import pickle

In [ ]:
### DO NOT EDIT ###

if __name__=='__main__':
    from google.colab import drive
    drive.mount('/content/drive')
    print()
    if rnn_encoder is not None and rnn_encoder is not None:
        print("Saving RNN model....")
        torch.save(rnn_encoder, 'drive/My Drive/rnn_encoder.pt')
        torch.save(rnn_decoder, 'drive/My Drive/rnn_decoder.pt')
    if transformer_encoder is not None and transformer_decoder is not None:
        print("Saving Transformer model....")
        torch.save(transformer_encoder, 'drive/My Drive/transformer_encoder.pt')
        torch.save(transformer_decoder, 'drive/My Drive/transformer_decoder.pt')

Mounted at /content/drive

Saving RNN model....
Saving Transformer model....
